# Part 1

## One to Many

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
data=pd.read_csv('wine-quality/data.csv',sep=';')

In [61]:
def normalize(data,list_attribute,thershold):
    norm = (data - data.mean())/data.std()
    norm['quality']=data['quality']
    return norm

In [62]:
list_attribute= data.columns[:-1]
# print list_attribute
data=normalize(data,list_attribute,0.5)
li=[1]*len(data.index)
data.insert(0, "Extra", li) 
train_data=data.sample(frac=0.8)
validation_data=data.drop(train_data.index)

In [63]:
def sigmod_function(a):
    return 1/(1+np.exp(-a))

In [64]:
def gradiant_method_mean_sq_error(X,Y,beta,learning_rate,m):
#     print X.shape
    a=np.dot(X,beta)
#     print Y.shape
    a=sigmod_function(a)
    z=np.subtract(a,Y)
#     print z.shape
    a=(learning_rate/m)*(np.dot(z.T,X))
#     print a.shape
    beta=np.subtract(beta,a.T)
#     print beta.shape
    return beta
def calc_predication(beta,test_data):
    predication=sigmod_function(np.dot(test_data.iloc[:,:-1].values,beta))
    return predication
def value_transformation(predication,thershold):
    li=[]
    for i in predication:
        if i[0]>=thershold:
            li.append(1)
        else:
            li.append(0)
    return li


In [65]:
def output_transfrom(Y,value):
    li=[]
    for i in Y:
        if i == value:
            li.append(1)
        else :
            li.append(0)
    return li

In [66]:
n=len(train_data.columns)-1

In [67]:
dict_beta={}
predication={}
output_label='quality'
# print beta.shape
for i in range(0,11):
    dict_beta[i]=np.array([0]*n)
#     X=train_data.iloc[:,:-1].values
#     Y=num.array(train_data['Chance of Admit '].tolist())
    dict_beta[i]=dict_beta[i].reshape(n,1)
    X=train_data.iloc[:,:-1].values
    Y=np.array(train_data[output_label].tolist())
    Y=np.array(output_transfrom(Y,i))
    m=len(train_data.index)
    Y=Y.reshape(m,1)
#     print Y
    for _ in range(10000):
        dict_beta[i]=gradiant_method_mean_sq_error(X,Y,dict_beta[i],0.1,m)
# print dict_beta
    predication[i]=calc_predication(dict_beta[i],validation_data)
    predication[i]=predication[i].T[0]
final_predication=[]
for i in range(len(validation_data.index)):
    max_v=0.0
    pos=0
    for j in range(0,11):
        if predication[j][i]>max_v:
            max_v=predication[j][i]
            pos=j
    final_predication.append(pos)
print "Accurary : "+str(accuracy_score(validation_data[output_label].tolist(),final_predication))

Accurary : 0.5374149659863946


# Part 2

## One to One

In [68]:
def logistic_ovo(X, Y, beta):
    Y.shape = (len(Y), 1)
#     print len(beta)
    beta.shape = (num_of_classes+1, 1)
    num_of_iters = 1000
    alpha = 0.001
    for _ in range(num_of_iters):
        z = np.dot(X , beta)
        hypo = sigmod_function(z)
        loss = np.subtract(hypo, Y)
        gradient = (alpha/len(Y)) * np.dot(X.T , loss)
        beta -= gradient
    beta.shape = (num_of_classes+1, )
    return beta

In [71]:
def predict_ovo(test_data):
    pred = []
    for index, row in test_data.iterrows():
        row_wise_pred = [0]* num_of_classes
        row = row.values
        for c in range(len(classifiers)):
            classifiers[c].shape = (num_of_classes+1, 1)
            row.shape = (num_of_classes+1, 1)
#             print(row.shape, classifier.T.shape)
            x=sigmod_function(np.dot(row.T , classifiers[c]))
            if x < 0.5:
                row_wise_pred[pair_list[c][0]] += 1
            else:
                row_wise_pred[pair_list[c][1]] += 1
#         print row_wise_pred
        pred.append(np.argmax(row_wise_pred))
    return pred

In [72]:
data = pd.read_csv('wine-quality/data.csv',sep=';')
list_attribute=data.columns[:-1]
data=normalize(data,list_attribute,0.5)
data_train = data.sample(frac = 0.8, random_state = 200)
data_validation = data.drop(data_train.index)
num_of_classes = 11
x = data_train.iloc[:, :-1].values
ones = np.ones(len(data_train))
X = np.concatenate((ones[:, np.newaxis], x), axis = 1)
unique_classes = data_train['quality'].unique()
label_train=data_train['quality']
label_validation=data_validation['quality']
pair_list = []
for i in range(len(unique_classes)):
    for j in range(i+1, len(unique_classes)):
        pair_list.append([unique_classes[i], unique_classes[j]])        
num_of_classifiers = len(pair_list)
classifiers = np.zeros(shape = (num_of_classifiers, len(data.keys())))
for classifier in range(num_of_classifiers):
    xtemp1 = X[label_train == pair_list[classifier][0]]
    xtemp2 = X[label_train == pair_list[classifier][1]]
    ytemp1 = label_train[label_train == pair_list[classifier][0]]
    ytemp2 = label_train[label_train == pair_list[classifier][1]]
    temp_X = np.concatenate((xtemp1, xtemp2))
    temp_Y = np.concatenate((ytemp1, ytemp2))
    classifiers[classifier, :] = logistic_ovo(temp_X, temp_Y, np.array(classifiers[classifier, :]))
predicted_ovo = predict_ovo(wine_validation)
print "Accuracy : "+str(accuracy_score(label_validation.tolist(),predicted_ovo))

Accuracy : 0.2471655328798186
